# 0. imports

In [13]:
# !pip install sqlalchemy sqlite3 pandas

In [14]:
import sqlite3
import pandas as pd

# 1. data

In [15]:
charms = pd.read_csv('../01_data/charms.csv', delimiter=';')  
iaps = pd.read_csv('../01_data/iaps.csv', delimiter=';')

# Step 2: Create a connection to the SQLite database
db_path = '../02_db/my_database.db'  # Replace with your SQLite database path
conn = sqlite3.connect(db_path)

# Write data into tables
charms.to_sql('charms', conn, if_exists='replace', index=False)
iaps.to_sql('iaps', conn, if_exists='replace', index=False)

13

In [16]:
charms.columns

Index(['DATE', 'SERVER_ID', 'SETS_COMPLETED_LIFETIME'], dtype='object')

In [17]:
iaps.columns

Index(['DATE', 'SERVER_ID', 'PAY'], dtype='object')

In [18]:
charms.columns

Index(['DATE', 'SERVER_ID', 'SETS_COMPLETED_LIFETIME'], dtype='object')

# 2. sql

## Q1

1. How many users have finished the event?
The goal of this task is to find how many users (identified by their SERVER_ID) have completed exactly 18 sets within the event period (from 5th December 2019 to 2nd January 2020).

Query Description:

This query selects distinct SERVER_IDs where the number of SETS_COMPLETED_LIFETIME is exactly 18, indicating they have completed the event.
The DATE range is between the start and end date of the event (5th December 2019 to 2nd January 2020).

In [25]:
task_1_query = """
SELECT COUNT(DISTINCT SERVER_ID) AS users_finished_event
FROM charms
WHERE SETS_COMPLETED_LIFETIME = 18
AND DATE BETWEEN '2019-12-05' AND '2020-01-02';
"""
result_1 = pd.read_sql_query(task_1_query, conn)
print("Task 1: Users who finished the event:")
print(result_1)

Task 1: Users who finished the event:
   users_finished_event
0                     0


## Q2

2. What is the average amount of $ spent to reach the end of the event?
In this task, we need to compute the average amount spent by users to reach 18 sets, excluding any purchases made after completing the 18th set (e.g., after 20th December 2019).

Query Description:

This query calculates the total amount spent by users who completed 18 sets and were active up to 20th December 2019.
We aggregate the PAY column by SERVER_ID and filter purchases that were made up to the completion of 18 sets.

In [ ]:
task_2_query = """
WITH completed_sets AS (
    SELECT SERVER_ID, MAX(SETS_COMPLETED_LIFETIME) AS max_sets
    FROM charms
    WHERE DATE <= '2019-12-20'
    GROUP BY SERVER_ID
)
SELECT AVG(iaps.PAY) AS avg_spent
FROM iaps
JOIN completed_sets ON iaps.SERVER_ID = completed_sets.SERVER_ID
WHERE completed_sets.max_sets = 18;
"""
result_2 = pd.read_sql_query(task_2_query, conn)
print("Task 2: Average amount spent to reach the end of the event:")
print(result_2)

Task 2: Average amount spent to reach the end of the event:
  avg_spent
0      None


## Q3

3. For each player, calculate cumulative revenue per set completed.
Here, we need to calculate how much revenue each user has generated at each set level, with the revenue accumulating as the player progresses.

Query Description:

This query calculates the cumulative revenue for each player for each set completed. We join the charms table with the iaps table to get the revenue data per player.
The revenue is calculated cumulatively for each set completed.

In [26]:
task_3_query = """
WITH cumulative_revenue AS (
    SELECT charms.SERVER_ID, charms.SETS_COMPLETED_LIFETIME, SUM(iaps.PAY) AS total_revenue
    FROM charms
    JOIN iaps ON charms.SERVER_ID = iaps.SERVER_ID
    WHERE charms.SETS_COMPLETED_LIFETIME <= 18
    GROUP BY charms.SERVER_ID, charms.SETS_COMPLETED_LIFETIME
)
SELECT SERVER_ID, SETS_COMPLETED_LIFETIME, total_revenue
FROM cumulative_revenue
ORDER BY SERVER_ID, SETS_COMPLETED_LIFETIME;
"""
result_3 = pd.read_sql_query(task_3_query, conn)
print("Task 3: Cumulative revenue per set completed:")
print(result_3)


Task 3: Cumulative revenue per set completed:
Empty DataFrame
Columns: [SERVER_ID, SETS_COMPLETED_LIFETIME, total_revenue]
Index: []


## Q4

4. We want to extend the event for 2 more days. What strategy would you recommend to increase revenue?
To extend the event and increase revenue, the strategy would involve targeting players who are close to completing the event or those who made significant purchases but haven't yet reached the maximum sets.

SQL for Strategy:

A strategy could be to offer additional incentives to players who are near completion of the sets or those who have spent significant amounts during the event.
We can identify players who made purchases in the last few days and offer them extra incentives.

In [28]:
task_4_query = """
SELECT SERVER_ID, SUM(iaps.PAY) AS total_spent
FROM iaps
WHERE SERVER_ID IN (SELECT SERVER_ID FROM charms WHERE SETS_COMPLETED_LIFETIME = 17)
AND DATE BETWEEN '2019-12-21' AND '2019-12-22'
GROUP BY SERVER_ID
HAVING total_spent > 10;
"""
result_4 = pd.read_sql_query(task_4_query, conn)
print("Task 4: Players who made purchases just before event ends:")
print(result_4)

Task 4: Players who made purchases just before event ends:
Empty DataFrame
Columns: [SERVER_ID, total_spent]
Index: []


## Q5

5. Can we say the 20% increase in revenue is due to the success of the feature?
This task requires analyzing the revenue growth during the event period and comparing it to the revenue from previous periods.

Query Description:

This query compares the total revenue before and during the event to determine if the 20% increase can be attributed to the event feature.

In [29]:
task_5_query = """
WITH pre_event_revenue AS (
    SELECT SUM(PAY) AS total_revenue_pre_event
    FROM iaps
    WHERE DATE BETWEEN '2019-11-01' AND '2019-12-04'
),
event_revenue AS (
    SELECT SUM(PAY) AS total_revenue_event
    FROM iaps
    WHERE DATE BETWEEN '2019-12-05' AND '2020-01-02'
)
SELECT 
    total_revenue_pre_event,
    total_revenue_event,
    (total_revenue_event - total_revenue_pre_event) / total_revenue_pre_event * 100 AS revenue_increase_percentage
FROM pre_event_revenue, event_revenue;
"""
result_5 = pd.read_sql_query(task_5_query, conn)
print("Task 5: Revenue increase during the event:")
print(result_5)

Task 5: Revenue increase during the event:
  total_revenue_pre_event total_revenue_event revenue_increase_percentage
0                    None                None                        None


In [11]:
# conn.close()